In [1]:
import pandas as pd
import syft as sy
import numpy as np
from syft.core.adp.entity import DataSubject

### Loading the dataset

In [2]:
data = pd.read_csv("../datasets/ca - feb 2021.csv")[0:10]
data.head()

/Users/areopagus/opt/anaconda3/envs/syft/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Classification,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter Code,Reporter,...,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit Code,Qty Unit,Qty,Netweight (kg),Trade Value (US$),Flag
0,HS,2021,202102,February 2021,4,0,1,Imports,124,Canada,...,"Other Asia, nes",NaN,6117,"Clothing accessories; made up, knitted or croc...",0,NaN,NaN,NaN,9285,0
1,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Egypt,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,116604,0
2,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,United Kingdom,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,1495175,0
3,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,United Rep. of Tanzania,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,2248,0
4,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Singapore,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,47840,0


### Logging into the domain

In [6]:
# Let's login into the domain
ca = sy.login(email="info@openmined.org", password="changethis", port=8081)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to http://localhost:8081... done! 	 Logging into musing_kearns... done!


### Upload the dataset to Domain node

In [7]:
# We will upload only the first few rows
# All these three columns are of `int` type

# NOTE: casting this tensor as np.int32 is REALLY IMPORTANT. We need to create flags for this or something
canada_trade = ((np.array(list(data['Trade Value (US$)'])) / 100000)[0:10]).astype(np.int32)
trade_partners = ((list(data['Partner'])))[0:10]

entities = list()
for i in range(len(trade_partners)):
    entities.append(DataSubject(name=trade_partners[i]))

# Upload a private dataset to the Domain object, as the root owner
sampled_canada_dataset = sy.Tensor(canada_trade)
sampled_canada_dataset.public_shape = sampled_canada_dataset.shape
sampled_canada_dataset = sampled_canada_dataset.private(0, 3, entities="bob").tag("trade_flow")

In [8]:
ca.load_dataset(
    assets={"Canada Trade": sampled_canada_dataset},
    name="Canada Trade Data - First few rows4",
    description="""A collection of reports from Canada's statistics 
                    bureau about how much it thinks it imports and exports from other countries.""",
)

Loading dataset... uploading... SUCCESS!                                                                                                                                      

Run <your client variable>.datasets to see your new dataset loaded into your machine!


In [9]:
ca.datasets

Idx,Name,Description,Assets,Id
[0],Canada Trade Data - First few rows4,A collection of reports from Canada's statistics bureau about how much it thinks it imports and exports from other countries.,"[""Canada Trade""] -> Tensor",73a1a7b8-11a9-466f-a1c0-91f54e5c6ecd


### Create a Data Scientist User

In [10]:
ca.users.create(
    **{
        "name": "Sheldon Cooper",
        "email": "sheldon@caltech.edu",
        "password": "bazinga",
        "budget":200
    }
)

In [11]:
ca.users

,added_by,allocated_budget,budget,budget_spent,created_at,daa_pdf,email,id,institution,name,role,verify_key,website
0,<syft.lib.python._SyNone object at 0x7ffd01263...,0.0,5.55,0.0,2021-11-05 01:53:57.824103,<syft.lib.python._SyNone object at 0x7ffd01263...,info@openmined.org,1,<syft.lib.python._SyNone object at 0x7ffd01263...,Jane Doe,Owner,385c768d00385a002478480416b672e518f81f9137d8cf...,<syft.lib.python._SyNone object at 0x7ffd01263...
1,Jane Doe,0.0,200.00,0.0,2021-11-05 01:54:06.122866,1,sheldon@caltech.edu,2,,Sheldon Cooper,Data Scientist,1e465712b3d4c31fdf6ca02478e91f844afaceb57d5d46...,


### Accept/Deny Requests to the Domain

In [18]:
ca.requests.pandas

,Requested Object's tags,Reason,Request ID,Requested Object's ID,Requested Object's type
0,[],,<UID: c578a9f2ccac41ff86c07a6142c1ba97>,<UID: 0c3bac4e8cc8e13ac093a3146d57e517>,
1,"[#Canada Trade, __add__]",,<UID: 20c895209adf46888a88370270ae9131>,<UID: 3f8ee0053025414390f88197a902be73>,
2,[],,<UID: 987ebbf8420e4f75b0c968376e8ae4c0>,<UID: ee99fd7bf376bab9608b29241f4fe5f1>,
3,[],,<UID: 2af5ce39fc544e55aa5439d12b0ee20a>,<UID: ede062b301fea6fc877f271c18b1ac20>,


In [19]:
ca.requests[-1].accept()